# Standard coalescent

In [1]:
library(ggplot2)
library(viridis)
library(grid)
library(gridExtra)
library("IRdisplay")

Indlæser krævet pakke: viridisLite



R header:

In [2]:
library(tidyverse)
library(devtools)
#remove.packages("ptdalgorithms")
# devtools::install_github("TobiasRoikjer/PtDAlgorithms")
#devtools::install_local('../../PtDAlgorithms', quiet=FALSE)
devtools::load_all(path="../../PtDAlgorithms", quiet=FALSE, recompile=TRUE)

library(ptdalgorithms)

── Attaching core tidyverse packages ──────────────────────────────────────────────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.2     ✔ readr     2.1.4
✔ forcats   1.0.0     ✔ stringr   1.5.0
✔ lubridate 1.9.2     ✔ tibble    3.2.1
✔ purrr     1.0.1     ✔ tidyr     1.3.0
── Conflicts ────────────────────────────────────────────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::combine() masks gridExtra::combine()
✖ dplyr::filter()  masks stats::filter()
✖ dplyr::lag()     masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors
Indlæser krævet pakke: usethis

ℹ Loading ptdalgorithms


Exports from /Users/kmt/PtDAlgorithms/src/ptdalgorithms.cpp:
   SEXP create_graph(size_t state_length)
   List find_or_create_vertex(SEXP phase_type_graph, IntegerVector state)
   void add_edge(SEXP phase_type_vertex_from, SEXP phase_type_vertex_to, double weight, NumericVector parameterized_edge_state = NumericVector::create())
   void graph_update_weights_parameterized(SEXP phase_type_graph, NumericVector scalars)
   List vertices(SEXP phase_type_graph)
   IntegerMatrix states(SEXP phase_type_graph)
   int vertices_length(SEXP phase_type_graph)
   List vertex_at(SEXP phase_type_graph, int index)
   List starting_vertex(SEXP phase_type_graph)
   List edges(SEXP phase_type_vertex)
   SEXP create_vertex(SEXP phase_type_graph, IntegerVector state)
   SEXP find_vertex(SEXP phase_type_graph, IntegerVector state)
   List graph_as_matrix(SEXP phase_type_graph)
   SEXP matrix_as_graph(NumericVector IPV, NumericMatrix SIM, Nullable <NumericMatrix> rewards = R_NilValue)
   SEXP clone_graph(SEXP

ℹ Re-compiling ptdalgorithms (debug build)



── R CMD INSTALL ───────────────────────────────────────────────────────────────────────────────────────────────────────
─  installing *source* package ‘ptdalgorithms’ ... (514ms)
   ** using staged installation
   ** libs
   using C++ compiler: ‘clang version 16.0.6’
   using SDK: ‘MacOSX15.2.sdk’
   arm64-apple-darwin20.0.0-clang++ -std=gnu++17 -I"/Users/kmt/miniconda3/envs/phasetype/lib/R/include" -DNDEBUG  -I'/Users/kmt/miniconda3/envs/phasetype/lib/R/library/Rcpp/include' -D_FORTIFY_SOURCE=2 -isystem /Users/kmt/miniconda3/envs/phasetype/include -mmacosx-version-min=11.0 -I/Users/kmt/miniconda3/envs/phasetype/include   -I../api/cpp -I../api/c -Ic/ -Icpp/ -fPIC  -ftree-vectorize -fPIC -fPIE -fstack-protector-strong -O2 -pipe -stdlib=libc++ -fvisibility-inlines-hidden -fmessage-length=0 -isystem /Users/kmt/miniconda3/envs/phasetype/include -fdebug-prefix-map=/Users/runner/miniforge3/conda-bld/r-base-split_1689933325543/work=/usr/local/src/conda/r-base-4.3.1 -fdebug-prefix-map=/Users/

In [3]:
# remove.packages("ptdalgorithms")

In [4]:
# install.packages("remotes")
# remotes::install_github("Thell/RcppMP")

In [5]:
# # GNUMPLIB
# ./configure --prefix=$CONDA_PREFIX --enable-cxx


In [6]:
# !export LIBS="-L/Users/kmt/miniconda3/envs/phasetype/lib"
# !export CPPFLAGS="-I/Users/kmt/miniconda3/envs/phasetype/include"

In [7]:
Rcpp::sourceCpp("./cpp/coalescent.cpp")

In [8]:
source("plot_functions.R")

In [9]:
theme_set(theme_bw())

despine <- theme(panel.border = element_blank(), panel.grid.major = element_blank(),
panel.grid.minor = element_blank(), axis.line = element_line(colour = "black"),
                text=element_text(family="Arial"))

In [10]:
# p <- ggplot(data=iris, aes(x = Sepal.Length, y = Sepal.Width)) +
#       geom_point(aes(color=Species, shape=Species)) + despine

# ggplotly(width=600, height=400)

In [11]:
despine <- theme(panel.border = element_blank(), panel.grid.major = element_blank(),
    panel.grid.minor = element_blank(), axis.line = element_line(colour = "black"),
                text=element_text(size=17)) 

options(repr.plot.width=10, repr.plot.height=7)

# ggplot(data=iris, aes(x = Sepal.Length, y = Sepal.Width)) +
#       geom_point(aes(color=Species, shape=Species), size=5, alpha=0.5) + scale_color_brewer(palette="Set1") + despine

# Issue: after expectation is run, gam$SIM doubles, fix or tell user not to reuse graph_as_matrix(graph) as matrix result

```
graph <- standard_coalescent(4)
print(graph_as_matrix(graph))
expectation(graph)
print(graph_as_matrix(graph))
```

## Generate standard coalescent state space

If you want to generate the state space:

In [12]:
standard_coalescent <- function(n) {
    
    # n <- 4
      
    state_vector_length <- n + 1
    graph <- create_graph(state_vector_length)
    starting_vertex <- vertex_at(graph, 1)
    initial_state <- c(rep(0, n), 0)
    initial_state[1] <- n
    
    add_edge(
      starting_vertex,
      create_vertex(graph, initial_state),
      1
    )
    index <- 2
    
    while (index <= vertices_length(graph)) {
      vertex <- vertex_at(graph, index)
      
      # loop over all classes of lineages
      for (i in 1:n) {
        for (j in i:n) {
          state <- vertex$state
          
          # if same class, there need to be at least two to coalesce
          if (i == j) {
            if (state[i] < 2) {
              next;
            }
            # coal rate
            rate <- state[i] * (state[i] - 1) / 2
          } else {
            # else at least one in each class to coalesce
            if (state[i] < 1 || state[j] < 1) {
              next;
            }
            # number of combinations
            rate <- state[i] * state[j]
          }
          
          # copy state
          child_state <- state
          # update child state
          child_state[i] <- child_state[i] - 1
          child_state[j] <- child_state[j] - 1
          child_state[i+j] <- child_state[i+j] + 1

          add_edge(
              vertex,
              find_or_create_vertex(graph, child_state),
              rate, c(rate)
            )
        }
      }
          
      index <- index + 1
    }
    return(graph)
}
# states <- t(sapply(1:vertices_length(graph), function(index) vertex_at(graph, index)$state ))
# ipv <- graph_as_matrix(graph)$IPV
# sim <- graph_as_matrix(graph)$SIM

In [13]:
# state_space <- function(initial_state, fun, ...) {
    
#     graph <- create_graph(length(initial_state))
#     starting_vertex <- vertex_at(graph, 1)
#     add_edge(
#       starting_vertex,
#       create_vertex(graph, initial_state),
#       1
#     )
#     index <- 2

#     while (index <= vertices_length(graph)) {
#       print(c(index, vertices_length(graph)))
#       vertex <- vertex_at(graph, index)
#       state <- vertex$state

#       result <- fun(state, graph=graph, ...)

#       if (length(result$states) > 0) {
#         for (i in 1:length(result$states)) {
#             add_edge(
#               vertex,
#               find_or_create_vertex(graph, c(result$states[[i]])),
#               c(result$rates[[i]]), c(result$rates[[i]])
#             )
#         }
#       }
#       index <- index + 1
#    }
#    return(graph)
# }



# # just give the state_space function an initial state, 
# # a callback function and any arguments the callback function
# # that takes a "state" and "graph" argument and any other
# # arguments passed state_space:

# n <- 4
# initial_state = rep(0, n)
# initial_state[1] <- n

# graph <- state_space(initial_state, kingman, n)
# graph

------

Just checking if I can do a "first-row solve" that is faster than R's solve.

In [28]:
graph <- standard_coalescent(10)
gam <- graph_as_matrix(graph)
nr_states <- nrow(gam$states)
nr_states

[1] 41

In [29]:
# plot_graph(graph_as_matrix(graph), size=c(10, 10), align=TRUE, rainbow=TRUE,
#                fontsize=16, ranksep=1, nodesep=0.5)
         

## Compute expected residence time in each state

### R version, that works (on DAGs only like coalescent)

In [16]:
residence_times <- function(graph) {
    res <- rep(0, vertices_length(graph))
    res[1] <- expectation(graph)
    sca <- rep(0, vertices_length(graph))
    sca[1] <- 1
    start_idx <- starting_vertex(graph)$index
    for (vertex in vertices(graph)) {
        idx <- vertex$index
        pushed <- 0
        for (edge in edges(vertex)) {
            child_idx <- edge$child$index
            child_prob <- edge$weight / vertex$rate
            wt <- 1/vertex$rate * sca[idx]            
            # if (vertex$index == 1) {
            if (idx == start_idx) {
                wt <- 0
            } 
            if (length(edges(edge$child)) > 0) { # don't push to absorbing
                push <- (res[idx] - wt) * child_prob
                res[child_idx] <- res[child_idx] + push
                sca[child_idx] <- sca[child_idx] + sca[idx] * child_prob
                pushed <- pushed + push
            }
        }
        res[idx] <- res[idx] - pushed
    }
    return(res)
}
#residence_times(graph)

### C++ version that should work on graphs with cycles as well

In [43]:
expected_residence_time(graph)

[1]   0.000000000 190.000000000 137.178947368  64.980139892  10.781286550
  [6]  15.606401095   7.916339906   1.228616524   0.484848298 -10.517792515
 [11]  -0.131921960  -0.365888265   0.161843180   1.283327592 -19.161366292
 [16]  -4.602074664  -7.536527467  -0.707837992  -0.700639889   0.015420007
 [21]   2.026694277 -10.514025992  -5.746481622 -16.506865775  -5.958153165
 [26]  -2.938529671  -0.170418752  -0.370813818  -1.278763830  -0.496711149
 [31]  -0.006772555   0.958636276  -1.345898941  -2.392601323 -13.693815414
 [36] -11.341651293  -3.661705160  -1.143174281  -1.174492580  -5.568697909
 [41]  -1.708987347  -3.601108606  -0.252725825  -0.270183332  -0.256572334
 [46]  -0.008646113   0.206703236   0.736178558   0.102795472  -4.382258722
 [51]  -8.063767196  -1.885228914  -1.911434841  -1.231247932  -7.699480499
 [56]  -8.795994271  -7.598262774  -0.296614384  -1.123727788  -1.026051711
 [61]  -0.576072508  -0.008902347  -0.036354667  -0.035864076  -1.127383103
 [66]  -0.191237055  -0.089888425  -0.008167510   0.020127986   0.263463935
 [71]   0.313239603  -0.120728096  -1.826953138  -0.314117659  -1.175730555
 [76]  -0.523787064  -4.301557240 -10.809829976  -6.794405102  -4.143553112
 [81]  -3.712393423  -3.950503052  -1.133252050   0.003362364  -0.128591450
 [86]  -0.085775781  -0.587765145  -0.373132113  -1.865208240  -0.186547084
 [91]  -0.034473378  -0.257158712  -0.170632439  -0.037291243  -0.080154910
 [96]  -0.065452790  -0.035798754  -0.005093776   0.000487129   0.023805410
[101]   0.067098785   0.146973214   0.170991260   0.036335507  -0.095494100
[106]  -0.008986219  -1.599099129  -5.462341940  -3.427768832  -2.576937809
[111]  -1.322500127  -1.811909126  -2.403044137  -0.386080358  -6.137721994
[116]  -2.117149416  -0.598434253  -3.280057824  -1.814125444  -0.771859028
[121]  -4.093846075  -6.081563218  -9.438957858  -1.065998520  -0.189102506
[126]  -2.058625637  -2.518680451  -2.405782407  -2.590919731  -1.448993914
[131]  -0.270265111  -0.137256002  -0.285142805  -0.087216016  -0.487320871
[136]  -1.020912787  -0.193111314  -0.321231140  -0.314599332  -0.011488524
[141]  -0.003636790  -0.015039911  -0.300784796  -0.048372256  -1.321051358
[146]  -0.420301147  -0.450904063  -4.341907055  -2.015282957  -7.165677992
[151]  -3.708457817  -3.838571536  -0.408470210  -2.502778980  -3.060233231
[156]  -0.521096915  -2.230117940 -10.264881550 -12.443628452  -9.927333793
[161] -20.709910720  -7.519104781  -4.634215118  -1.979645689  -0.793338796
[166]  -2.768224286  -2.332675451  -0.589380896  -0.745631523  -3.448248205
[171]  -0.512298091  -0.411012698  -0.500762079  -0.135632014  -0.064280574
[176]  -1.012716253  -1.940599292  -6.388373012  -5.649909548  -1.847206606
[181]  -2.911671975  -5.640058658  -0.828429052  -0.481193350  -0.731006005
[186]  -0.565095859  -0.562313302  -0.317970853  -0.081938640  -0.055880837
[191]   0.000000000  -0.028247736   0.000000000   0.000000000   0.000000000
[196]   0.000000000   0.000000000   0.000000000   0.000000000   0.000000000
[201]   0.000000000   0.000000000   0.000000000   0.000000000   0.000000000
[206]   0.000000000   0.000000000   0.000000000   0.000000000   0.000000000
[211]   0.000000000   0.000000000   0.000000000   0.000000000   0.000000000
[216]   0.000000000   0.000000000   0.000000000   0.000000000   0.000000000
[221]   0.000000000   0.000000000   0.000000000   0.000000000   0.000000000
[226]   0.000000000   0.000000000   0.000000000   0.000000000   0.000000000
[231]   0.000000000   0.000000000   0.000000000   0.000000000   0.000000000
[236]   0.000000000   0.000000000   0.000000000   0.000000000   0.000000000
[241]   0.000000000   0.000000000   0.000000000   0.000000000   0.000000000
[246]   0.000000000   0.000000000   0.000000000   0.000000000   0.000000000
[251]   0.000000000   0.000000000   0.000000000   0.000000000   0.000000000
[256]   0.000000000   0.000000000   0.000000000   0.000000000   0.000000000
[261]   0.000000000   0.000000000   0.000000000   

## To make general first-row-solve, it should handle all kinds of matrices, not only subintensity matrices with non-negative values and nagative diagonal

---

In [17]:
# with 4x4 matrix
a <- c(2, 1, 3, 3) 
b <- c(5, 3, 5, 4) 
c <- c(6, 5, 9, 6)
d <- c(1, 3, 2, 2)
xyz <- rbind(a, b, c, d) 
print(xyz)
solve(a=xyz) 

  [,1] [,2] [,3] [,4]
a    2    1    3    3
b    5    3    5    4
c    6    5    9    6
d    1    3    2    2


a,b,c,d
-0.2702703,0.56756757,-0.18918919,-0.1621622
-0.3243243,0.08108108,-0.02702703,0.4054054
-0.1621622,-0.45945946,0.48648649,-0.2972973
0.7837838,0.05405405,-0.35135135,0.2702703


In [18]:
# with 4x4 matrix
a <- c(2, 1, 3, 3) 
b <- c(5, 3, 5, 4) 
c <- c(6, 5, 9, 6)
d <- c(1, 3, 2, 2)
xyz <- rbind(a, b, c, d) 
m <- matrix(0, nrow=4, ncol=4)
diag(m) <- 1
solve(a=xyz) %*% m #%*% rep(1, 4)


-0.2702703,0.56756757,-0.18918919,-0.1621622
-0.3243243,0.08108108,-0.02702703,0.4054054
-0.1621622,-0.45945946,0.48648649,-0.2972973
0.7837838,0.05405405,-0.35135135,0.2702703


---

In [19]:
gam <- graph_as_matrix(graph)

In [20]:
v <- residence_times(matrix_as_graph(gam$IPV, gam$SIM))
first_row_U_ptd <- v[2:(length(v)-1)]
first_row_U_solve <- -solve(gam$SIM)[1, ]
max(first_row_U_ptd - first_row_U_solve)

[1] 8.120674e-17

In [21]:
start <- proc.time()[3]
v <- residence_times(matrix_as_graph(gam$IPV, gam$SIM))
first_row_U_ptd <- v[2:(length(v)-1)]
proc.time()[3] - start
#first_row_U_ptd

elapsed 
  2.601

In [22]:
# start <- proc.time()[3]
# v <- accumulated_visiting_time(graph, 100)
# #v
# first_row_U_ptd_acc <- v[2:(length(v)-1)]
# proc.time()[3] - start

In [23]:
# (first_row_U_ptd - first_row_U_ptd_acc)/first_row_U_ptd_acc

In [24]:
# max(abs((first_row_U_ptd - first_row_U_ptd_acc)/first_row_U_ptd_acc))

In [25]:
start <- proc.time()[3]
first_row_U_solve <- -solve(gam$SIM)[1, ]
# first_row_U_solve
proc.time()[3] - start

elapsed 
  2.756

In [53]:
(first_row_U_ptd - first_row_U_solve)/first_row_U_solve

[1]  3.131176e-15 -9.640726e-15 -4.761035e-14  7.091495e-15 -1.697757e-14
  [6] -6.009386e-14 -6.303551e-15 -1.994970e-14 -2.941657e-14  2.059160e-14
 [11]  4.748675e-14  6.894509e-15  2.256936e-14 -2.327465e-15 -2.327465e-15
 [16] -2.327465e-15 -3.325123e-14 -3.325123e-14  1.930463e-15 -4.813621e-15
 [21]  1.775691e-14 -4.813621e-15 -9.928094e-15 -1.631283e-14  6.067168e-14
 [26]  1.360517e-14 -2.083674e-15  1.360517e-14  1.360517e-14  3.585145e-15
 [31]  1.303689e-14 -7.946295e-15 -7.946295e-15  4.469791e-16 -8.211172e-15
 [36]  3.973148e-16  1.303689e-14 -2.607378e-15 -1.862413e-14 -2.855700e-15
 [41] -4.882626e-14  1.291273e-14 -4.865554e-14 -4.865554e-14  7.682453e-16
 [46] -9.150655e-15  1.092616e-14  5.931342e-15  8.331194e-15  8.604348e-15
 [51]  1.073105e-14  2.608620e-14  2.731539e-15  2.731539e-15 -9.218944e-15
 [56]  8.331194e-15 -2.057759e-14 -9.218944e-15 -8.809213e-15  5.121636e-16
 [61]  2.581304e-14 -9.014079e-15 -1.587707e-14 -9.218944e-15 -1.562099e-14
 [66] -1.587707e-14  6.690136e-15  4.694833e-15  1.105894e-14 -4.928332e-15
 [71]  2.026975e-14  5.365523e-15  5.365523e-15 -2.543655e-14 -2.527757e-14
 [76] -1.564944e-14  5.365523e-15  1.544078e-14 -2.503911e-15 -2.185954e-15
 [81]  5.365523e-15 -2.364805e-15  1.544078e-14  1.544078e-14  1.544078e-14
 [86]  5.365523e-15  1.544078e-14  1.544078e-14  1.544078e-14  4.381844e-15
 [91] -2.295251e-15  1.544078e-14  4.225349e-15  1.544078e-14  4.381844e-15
 [96]  4.225349e-15 -1.075899e-14 -4.401406e-15 -5.790294e-15  1.430806e-15
[101] -5.580144e-15 -7.154031e-16  6.930467e-15  4.225349e-15  1.430806e-15
[106] -1.015872e-14  1.269840e-14 -1.001564e-15  2.062745e-14  2.432370e-14
[111]  1.475519e-15  1.609657e-15 -1.902972e-14  2.062745e-14  1.341381e-15
[116]  1.609657e-15  2.432370e-14 -5.790294e-15  1.430806e-15  4.225349e-15
[121]  1.287726e-14  1.609657e-15  1.251955e-15 -1.019449e-14 -1.580594e-14
[126] -1.564944e-14 -5.633799e-15  1.305611e-14  1.251955e-15 -5.320810e-15
[131] -1.533645e-14 -5.790294e-15  1.475519e-15 -1.580594e-14 -5.633799e-15
[136] -5.320810e-15 -5.633799e-15 -5.633799e-15 -1.584506e-14  5.121636e-16
[141] -1.902322e-15 -1.638923e-15 -1.755989e-15  5.121636e-16 -1.755989e-15
[146]  5.931342e-15  5.853298e-15 -9.131145e-15 -1.560879e-15  5.853298e-15
[151]  5.775254e-15  5.853298e-15 -1.112127e-14  7.682453e-16 -1.560879e-15
[156]  1.334552e-14  1.334552e-14  1.334552e-14  5.775254e-15 -1.404791e-14
[161] -1.404791e-14  1.334552e-14 -1.404791e-14  8.048284e-15 -1.755989e-15
[166]  5.853298e-15 -3.902199e-15 -1.902322e-15  5.853298e-15 -1.404791e-14
[171]  7.462955e-15 -1.424302e-14  8.048284e-15  8.048284e-15 -1.755989e-15
[176]  1.334552e-14 -1.560879e-15 -1.755989e-15  8.048284e-15  8.048284e-15
[181]  5.121636e-16 -1.638923e-15 -1.560879e-15  8.048284e-15  7.609287e-15
[186]  7.682453e-16 -1.316992e-15  7.462955e-15  7.682453e-16  8.048284e-15
[191]  5.121636e-16  7.682453e-16  7.682453e-16 -1.365769e-16  2.617725e-15
[196]  6.145963e-15 -4.666379e-15 -4.097308e-16  2.549436e-15  8.536059e-16
[201]  2.731539e-15  1.019775e-14 -4.097308e-16 -1.365769e-16 -9.150655e-15
[206] -1.365769e-16  8.536059e-16  8.558822e-15  2.549436e-15  8.331194e-15
[211]  2.731539e-15  2.549436e-15  2.549436e-15  2.731539e-15 -3.095744e-15
[216]  8.376719e-15  2.367334e-15  8.376719e-15 -3.095744e-15  6.828847e-16
[221]  3.286383e-15 -1.183667e-14  2.549436e-15 -9.150655e-15  8.558822e-15
[226] -3.459949e-15 -3.277847e-15  2.549436e-15 -8.877502e-15  8.376719e-15
[231] -1.206430e-14  8.376719e-15  2.617725e-15  6.145963e-15  8.376719e-15
[236]  2.367334e-15 -1.365769e-16 -8.877502e-15 -3.277847e-15 -5.007821e-15
[241] -3.095744e-15 -1.206430e-14 -1.195048e-14 -4.609472e-15  2.503911e-15
[246] -1.365769e-16 -3.095744e-15  2.731539e-15 -1.195048e-14 -1.206430e-14
[251]  8.536059e-16 -4.097308e-16 -3.277847e-15 -4.780193e-15 -1.206430e-14
[256]  3.414424e-16 -4.097308e-16 -1.206430e-14  3.414424e-16 -4.666379e-15
[261]  3.414424e-16  6.145963e-15  3.435764e-15 -1

In [54]:
max(abs((first_row_U_ptd - first_row_U_solve)/first_row_U_solve))

[1] 6.067168e-14